<a href="https://colab.research.google.com/github/Shuvo3128/BACKBENCHER-/blob/main/Demo_Script.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pandas numpy scikit-learn spacy nltk bs4 matplotlib seaborn wordcloud

In [3]:
# ============================
# 1. Import Libraries
# ============================
import pandas as pd
import numpy as np
import pickle
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout

# ============================
# 2. Load Data
# ============================
train_df = pd.read_csv("/content/drive/MyDrive/train_clean.csv")
test_df  = pd.read_csv("/content/drive/MyDrive/test_clean.csv")

X_train_text = train_df['review_clean']
X_test_text  = test_df['review_clean']
y_train = train_df['sentiment'].astype(int).values
y_test  = test_df['sentiment'].astype(int).values

# ============================
# 3. Tokenization & Padding
# ============================
MAX_VOCAB = 30000
MAX_LEN = 200

tokenizer = Tokenizer(num_words=MAX_VOCAB)
tokenizer.fit_on_texts(X_train_text)

X_train_seq = tokenizer.texts_to_sequences(X_train_text)
X_test_seq  = tokenizer.texts_to_sequences(X_test_text)

X_train_pad = pad_sequences(X_train_seq, maxlen=MAX_LEN)
X_test_pad  = pad_sequences(X_test_seq, maxlen=MAX_LEN)

# ============================
# 4. Build LSTM Model
# ============================
model = Sequential([
    Embedding(input_dim=MAX_VOCAB, output_dim=128, input_length=MAX_LEN),
    LSTM(64, return_sequences=False, dropout=0.4, recurrent_dropout=0.4),
    Dropout(0.5),
    Dense(32, activation='relu'),
    Dropout(0.3),
    Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# ============================
# 5. Train Model
# ============================
early_stop = EarlyStopping(monitor='val_loss', patience=2, restore_best_weights=True)
model.fit(
    X_train_pad, y_train,
    validation_split=0.2,
    epochs=5,
    batch_size=64,
    callbacks=[early_stop],
    verbose=1
)

# ============================
# 6. Save Model & Tokenizer
# ============================
model.save("lstm_sentiment_model.h5")
with open("tokenizer.pkl", "wb") as f:
    pickle.dump(tokenizer, f)

print("✅ Model and Tokenizer Saved!")


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/5
312/312 ━━━━━━━━━━━━━━━━━━━━ 223s 680ms/step - accuracy: 0.6591 - loss: 0.5982 - val_accuracy: 0.8324 - val_loss: 0.3752
Epoch 2/5
312/312 ━━━━━━━━━━━━━━━━━━━━ 258s 686ms/step - accuracy: 0.8850 - loss: 0.3039 - val_accuracy: 0.8466 - val_loss: 0.3560
Epoch 3/5
312/312 ━━━━━━━━━━━━━━━━━━━━ 260s 682ms/step - accuracy: 0.9118 - loss: 0.2459 - val_accuracy: 0.8378 - val_loss: 0.3848
Epoch 4/5
312/312 ━━━━━━━━━━━━━━━━━━━━ 214s 687ms/step - accuracy: 0.9336 - loss: 0.1907 - val_accuracy: 0.8526 - val_loss: 0.4221


✅ Model and Tokenizer Saved!


In [5]:
import pickle
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.sequence import pad_sequences

# ============================
# 1. Load Model & Tokenizer
# ============================
model = load_model("/content/lstm_sentiment_model.h5")
with open("/content/tokenizer.pkl", "rb") as f:
    tokenizer = pickle.load(f)

MAX_LEN = 200  # Must match training

# ============================
# 2. Prediction Function
# ============================
def predict_sentiment(text):
    seq = tokenizer.texts_to_sequences([text])
    pad_seq = pad_sequences(seq, maxlen=MAX_LEN)
    prob = model.predict(pad_seq)[0][0]
    sentiment = "Positive 😀" if prob > 0.5 else "Negative 😠"
    return sentiment, prob

# ============================
# 3. Interactive Demo Loop
# ============================
print("=== Sentiment Analysis Demo (LSTM) ===")
while True:
    user_input = input("\nEnter a review (or 'quit' to exit): ")
    if user_input.lower() == 'quit':
        print("👋 Goodbye!")
        break
    sentiment, prob = predict_sentiment(user_input)
    print(f"Predicted Sentiment: {sentiment}  |  Confidence: {prob:.4f}")


=== Sentiment Analysis Demo (LSTM) ===

Enter a review (or 'quit' to exit): This film is absolutely awful, but nevertheles
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 502ms/step
Predicted Sentiment: Negative 😠  |  Confidence: 0.2580

Enter a review (or 'quit' to exit):  I got to see this film at a preview and was da
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 137ms/step
Predicted Sentiment: Positive 😀  |  Confidence: 0.9108

Enter a review (or 'quit' to exit): This adaptation positively butchers a classic 
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 137ms/step
Predicted Sentiment: Positive 😀  |  Confidence: 0.7813

Enter a review (or 'quit' to exit): quit
👋 Goodbye!
